# Example of using pandasql library for data analysis

In [1]:
%matplotlib inline
import pandas as pd
import pandasql as ps
from datetime import datetime
import seaborn
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

In [2]:
pd.__version__

'0.24.1'

In [3]:
project_submissions = pd.read_csv('./data/project_submissions.csv')
daily_engagements = pd.read_csv('./data/daily_engagement.csv')
enrollments = pd.read_csv('./data/enrollments.csv')



In [4]:
project_submissions.head()

,creation_date,completion_date,assigned_rating,account_key,lesson_key,processing_state
0,2015-01-14,2015-01-16,UNGRADED,256,3176718735,EVALUATED
1,2015-01-10,2015-01-13,INCOMPLETE,256,3176718735,EVALUATED
2,2015-01-20,2015-01-20,PASSED,256,3176718735,EVALUATED
3,2015-03-10,2015-03-13,PASSED,434,3176718735,EVALUATED
4,2015-02-17,2015-03-03,INCOMPLETE,434,3176718735,EVALUATED


In [5]:
daily_engagements.head()

,acct,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,2015-01-09,1.0,11.679374,0.0,0.0
1,0,2015-01-10,2.0,37.284887,0.0,0.0
2,0,2015-01-11,2.0,53.633746,0.0,0.0
3,0,2015-01-12,1.0,33.489270,0.0,0.0
4,0,2015-01-13,1.0,64.779678,0.0,0.0


In [6]:
enrollments.head()

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled
0,448,canceled,2014-11-10,2015-01-14,65.0,True,True
1,448,canceled,2014-11-05,2014-11-10,5.0,True,True
2,448,canceled,2015-01-27,2015-01-27,0.0,True,True
3,448,canceled,2014-11-10,2014-11-10,0.0,True,True
4,448,current,2015-03-10,NaN,NaN,True,False


## Простой SQL запрос 
Получить аккаунты и дату, с максимальным временем визита

In [7]:
# pandasql code
def example1_pandasql(daily_engagements):
    simple_query = '''
        SELECT 
            acct, 
            total_minutes_visited,
            utc_date
        FROM daily_engagements 
        ORDER BY total_minutes_visited desc
        LIMIT 5
        '''
    return ps.sqldf(simple_query, locals())

# pandas code
def example1_pandas(daily_engagements):
    return daily_engagements[['acct', 'total_minutes_visited', 'utc_date']].sort_values(by ='total_minutes_visited', ascending = False)[:5]

In [8]:
example1_pandasql(daily_engagements)

,acct,total_minutes_visited,utc_date
0,317,1030.883197,2015-07-11
1,328,945.538914,2015-07-09
2,198,876.512846,2014-12-30
3,163,872.633923,2015-07-10
4,573,866.405226,2015-07-11


In [9]:
example1_pandas(daily_engagements)

,acct,total_minutes_visited,utc_date
54536,317,1030.883197,2015-07-11
56403,328,945.538914,2015-07-09
33728,198,876.512846,2014-12-30
27699,163,872.633923,2015-07-10
97492,573,866.405226,2015-07-11


# Соединение таблиц


In [10]:
# pandasql code
def example3_pandasql(enrollments, daily_engagements):
    join_query = '''
        SELECT acct,status,sum(num_courses_visited) as num_courses_visited FROM daily_engagements JOIN enrollments 
        ON acct = account_key GROUP BY acct ORDER BY acct ASC LIMIT 10
    '''
    return ps.sqldf(join_query, locals()).set_index('status')

# pandas code
def example3_pandas(enrollments, daily_engagements):
    join_df = pd.merge(daily_engagements, 
                   enrollments[enrollments.is_udacity == 0], 
                   how = 'inner', 
                   right_on ='account_key', 
                   left_on = 'acct')
    join_df = join_df[['account_key', 'status', 'num_courses_visited']]
    #avg_account_total_minutes = pd.DataFrame(join_df.groupby(['account_key', 'status'], as_index = False)
                                                     #.total_minutes_visited.mean())
    #avg_total_minutes= pd.DataFrame(avg_account_total_minutes.groupby('status').total_minutes_visited.mean())
    #avg_total_minutes.columns = ['avg_total_minutes']
    res = join_df.groupby('account_key')
    q = res.agg({'status':'first','num_courses_visited':'sum'})[:10]
    return pd.DataFrame(q)#avg_total_minutes

In [11]:
example3_pandasql(enrollments, daily_engagements)

,acct,num_courses_visited
status,,
current,0,108.0
current,1,52.0
current,2,39.0
current,3,118.0
current,4,40.0
canceled,5,144.0
current,6,55.0
current,7,36.0
current,8,71.0


In [12]:
example3_pandas(enrollments, daily_engagements)

,status,num_courses_visited
account_key,,
0,current,108.0
1,current,52.0
2,current,39.0
3,canceled,118.0
4,current,40.0
5,canceled,144.0
6,current,55.0
7,current,36.0
8,current,71.0


# Вычисление времени выполнения

In [13]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start

## Сравнение

In [14]:
with Timer() as t:
    example3_pandasql(enrollments, daily_engagements)

print('pandasql: %.03f sec.' % t.interval)

with Timer() as t:
    example3_pandas(enrollments, daily_engagements)

print('pandas: %.03f sec.' % t.interval)

pandasql: 2.945 sec.
pandas: 0.217 sec.
